# Projekt 1.

## Przygotowanie danych.

Domyślnie wczytuję dane ze zmiennymi które arbitralnie sobie wybrałem - zmienne kategoryczne przekształcone do macierzy zer i jedynek (one-hot encoding). Czyli nie ma zmiennej "linia lotnicza", ale jest zmienna "Is_Ryanair", "Is_LOT" - itp., zmienne binarne. Oczywiście zwykłe numeryczne też są. 

In [2]:
import pandas as pd
import numpy as np

# ostateczne dane do przewidywania ceny
dane = pd.read_pickle("dane_onehot.pkl")
#dane = pd.read_pickle("Indian_Dataset.pkl")

dane.head()

,Departure_time,Arrival_time,Flight_time,Price,Num_Layovers,Cabin_bag,Checked_bag,Days_to_departure,layover_duration,Is_Ekonomiczna,...,Is_Wizz Air1,Is_easyJet1,Is_Air France2,Is_British Airways2,Is_KLM2,Is_LOT2,Is_Lufthansa2,Is_Ryanair2,Is_Wizz Air2,Is_easyJet2
0,16.58,17.92,1.33,1643,0,1,0,2,0.000000,True,...,False,False,False,False,False,False,False,False,False,False
1,18.33,22.67,4.33,1087,1,0,0,2,1.583333,True,...,True,False,False,False,False,False,False,False,False,True
2,19.67,21.00,1.33,1749,0,1,0,2,0.000000,True,...,False,False,False,False,False,False,False,False,False,False
3,6.08,11.92,5.83,1776,1,0,0,2,1.083333,True,...,True,False,False,False,False,False,False,False,False,True
4,17.00,22.67,5.67,1878,1,1,0,2,2.833333,True,...,False,False,False,False,False,False,True,False,False,False


In [3]:
from sklearn.model_selection import train_test_split

# przekształcamy na numeryczne (na wszelki wypadek)
dane = dane.apply(pd.to_numeric)

# target variable - cena lotu (zł)
X = dane.drop(columns = "Price")
y = dane["Price"]

# podział na zbiór treningowy, walidacyjny i testowy - proporcje 80% : 20% : 20% 
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.75)

# konwersja do macierzy numpy
X_train_np = X_train.to_numpy().astype(np.float64)
y_train_np = y_train.to_numpy().astype(np.float64)
X_val_np = X_val.to_numpy().astype(np.float64)
y_val_np = y_val.to_numpy().astype(np.float64)
X_test_np = X_test.to_numpy().astype(np.float64)
y_test_np = y_test.to_numpy().astype(np.float64)


In [4]:
X.shape

(144313, 37)

## Testowanie parametrów

Grid Search - przeszukiwanie kombinacji parametrów. 

In [22]:
import itertools
from glob import glob
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from perceptron_nn import MultiLayerPerceptron

num_repetitions =5
baseline_params = {
    "num_inputs": [X_test_np.shape[1]], # tego nie ruszać
    "num_layers": [
        [2, 3, 4],
        [2, 1, 2]
    ],
    "learning_rate": [0.1],
    "num_epochs": [20000],
    "activation_function": ["relu"]
}


# param_dict = {
#     "num_inputs": [37],  # liczba neuronów w warstwie wejściowej
#     "num_layers": [
#         # Architektury z jedną warstwą ukrytą (mniejszą niż wejściowa)
#         [30], [20], [15], [10],
        
#         # Architektury z jedną warstwą ukrytą (podobną do wejściowej)
#         [37], [40],
        
#         # Architektury z kilkoma malejącymi warstwami ukrytymi
#         [30, 20], [37, 18], [30, 15, 8], [37, 25, 12],
        
#         # Architektury z sukcesywnie malejącą liczbą neuronów
#         np.arange(35, 5, -5).tolist(),     # [35, 30, 25, 20, 15, 10]
#         np.arange(37, 7, -6).tolist(),     # [37, 31, 25, 19, 13, 7]
#         np.arange(30, 0, -3).tolist(),     # [30, 27, 24, 21, 18, 15, 12, 9, 6, 3]
        
#         # Architektury typu "piramida" (coraz mniejsze)
#         [37, 30, 25, 20, 15, 10], [25, 20, 15, 10, 5]
#     ],
#     "learning_rate": [0.1, 0.05],
#     "num_epochs": [2000],
#     "activation_function": ["tanh", "relu"]
# }

keys = list(baseline_params.keys())
combinations = list(itertools.product(*(baseline_params[key] for key in keys)))

# ramka danych z parametrami
params_df = pd.DataFrame(data = combinations, columns = keys)

# pusta lista na wyniki
results = []
for i, row in params_df.iterrows():
    try:
    
        num_inputs = row["num_inputs"]
        num_layers = row["num_layers"]
        learning_rate = row["learning_rate"]
        activation_function = row["activation_function"]
        num_epochs = row["num_epochs"]
        

        # żeby sprawdzić wszystkie parametry metody fit, idź do perceptron_nn.ipynb
        epochs = []
        train_mse = []
        val_mse = []
        test_mse = []
        test_mae = []
        test_mape = []
        test_r2 = []
        for k in range(num_repetitions):
            print(f"{k} powtórzenie")
            mlp = MultiLayerPerceptron(
            num_inputs= num_inputs, num_layers=num_layers, learning_rate=learning_rate, 
            activation_function=activation_function)

            mlp.fit(
                X=X_train_np,
                y=y_train_np,
                X_val=X_val_np,
                y_val=y_val_np,
                num_epochs=num_epochs,
                verbose=True)
            
            res = mlp.history
            pred = mlp.predict(X_test_np)
            epochs.append(res["epoch"])
            train_mse.append(res["train_mse"])
            test_mse.append(mean_squared_error(pred, y_test_np))
            val_mse.append(res["val_mse"])
            test_mae.append(mean_absolute_error(pred, y_test_np))
            test_mape.append(mean_absolute_percentage_error(pred, y_test_np))
            test_r2.append(r2_score(y_test_np, pred))

        results.append(
            {"num_inputs": num_inputs, 
            "num_layers": num_layers,
            "learning_rate": learning_rate,
            "num_epochs": epochs,
            "activation_function": activation_function,
            "train_MSE": sum(train_mse)/len(train_mse),
            "val_MSE": sum(val_mse)/len(val_mse),
            "test_MSE": sum(test_mse)/len(test_mse),
            "test_MAE": sum(test_mae)/len(test_mae),
            "test_MAPE": sum(test_mape)/len(test_mape),
            "test_R2": sum(test_r2)/len(test_r2)
            }
        )
    except KeyboardInterrupt:
        break


result_df = pd.DataFrame(results)

files = glob("nn_*.xlsx")
# zapis wyników do pliku
result_df.to_excel(f"nn_{len(files)+1}.xlsx")

0 powtórzenie
Architektura sieci: Wejścia(37) -> Warstwa_1(2) -> Warstwa_2(3) -> Warstwa_3(4) -> Wyjście(1)
Epoka 100/20000, Train MSE: 879357.4217, Val MSE: 896645.3923
Epoka 200/20000, Train MSE: 695740.6685, Val MSE: 720462.8005
Epoka 300/20000, Train MSE: 569842.3904, Val MSE: 623387.9967
Epoka 400/20000, Train MSE: 515065.6551, Val MSE: 546104.4601
Epoka 500/20000, Train MSE: 446118.9704, Val MSE: 486660.7534
Epoka 600/20000, Train MSE: 429320.7366, Val MSE: 437295.6757
Early stopping na epoce 676/20000. 50 epok bez poprawy. Najlepszy val_MSE: 427472.9571
1 powtórzenie
Architektura sieci: Wejścia(37) -> Warstwa_1(2) -> Warstwa_2(3) -> Warstwa_3(4) -> Wyjście(1)
Epoka 100/20000, Train MSE: 1072065.1624, Val MSE: 1047745.2246
Early stopping na epoce 128/20000. 50 epok bez poprawy. Najlepszy val_MSE: 1047745.2246
2 powtórzenie
Architektura sieci: Wejścia(37) -> Warstwa_1(2) -> Warstwa_2(3) -> Warstwa_3(4) -> Wyjście(1)
Epoka 100/20000, Train MSE: 883526.1998, Val MSE: 817375.7608
Ear

In [23]:
results

[{'num_inputs': 37,
  'num_layers': [2, 3, 4],
  'learning_rate': 0.1,
  'num_epochs': [676, 128, 160, 657, 895],
  'activation_function': 'relu',
  'train_MSE': np.float64(629495.916671288),
  'val_MSE': np.float64(615625.2155781093),
  'test_MSE': 631067.1731705727,
  'test_MAE': 518.8270237062031,
  'test_MAPE': 0.5667683386483383,
  'test_R2': 0.4201918508838757}]